In [1]:
import spacy
import pickle
import pandas as pd

In [2]:
with open('pickleObjects/corefDictObject', 'rb') as f:
    corefAnnot = pickle.load(f)

with open('pickleObjects/speakersTokObject', 'rb') as f:
    speakersTok = pickle.load(f)

In [3]:
sweCorp = ''

with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-sv.txt') as sweDoc:
    for line in sweDoc:
        sweCorp = sweCorp + line

In [4]:
nlp = spacy.load('../Models/SwedishModel')
doc = nlp(sweCorp)

In [5]:
df2 = pd.DataFrame({'DocName': 'ep-09-03-11-018',
                    'DocID': pd.Series(0, index = list(range(len(doc))), dtype='int32'),
                    'TokenID': 1,
                    'Text': '',
                    'Tag': '',
                    'Speaker': '',
                    'Corefs': '-',})

In [6]:
tokenIdDict = {}

for sent in doc.sents:
    idTok = 0
    for token in sent:
        key = str(token.i)
        tokenIdDict[key] = idTok
        if token.text != '\n':
            idTok += 1

In [7]:
for i in range(len(doc)):
    key = str(i)
    df2.at[i, 'TokenID'] = tokenIdDict[key] 
    

In [8]:
for i, token in enumerate(doc):
    df2.at[i, 'Text'] = token.text
    df2.at[i, 'Tag'] = token.tag_

In [9]:
tokIndex = 0
tokStartIndex = 0
for sent in doc.sents:
    for i, token in enumerate(sent):
        if speakersTok.get(tokIndex - i) is not None:
            df2.at[tokIndex, 'Speaker'] = speakersTok[tokIndex - i]
        tokIndex += 1

In [10]:
for i in range(len(doc)):
    key = str(i)
    if corefAnnot.get(key) is not None:
        df2.at[i, 'Corefs'] = corefAnnot[key]

In [11]:
df2 = df2[df2.Text != '\n'].reset_index(drop=True)

In [12]:
with open('Dataframes/ep-09-03-11-018', 'wb') as f:
    pickle.dump(df2, f)
